In [1]:
from openai import OpenAI
import pandas as pd
import os
import json
import logging
import time
import dotenv
import asyncio
from openai import AsyncOpenAI

dotenv.load_dotenv()

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("Tariff Analysis")


In [2]:
gcap_data = "/oak/stanford/groups/maggiori/GCAP/data/"
df = pd.read_parquet(f"{gcap_data}shared/ai_geo1/temp/transcripts/transcripts_2025_workshop_sample.parquet").sample(20)
df.head()

,key,date,companyname_primary,companyid_primary,gvkey_primary,companynames,companyids,gvkeys,version_date,year,file_path,transcript
35,1915209450,2025-02-05,"Golub Capital BDC, Inc.",81683203,[184689],"['Golub Capital BDC, Inc.']",[81683203],[[184689]],2025-04-01,2025,2025/1915209450.txt,"HEADLINE= Golub Capital BDC, Inc., Q1 2025 Ear..."
36,1940600983,2025-04-29,Entra ASA,254167079,[318538],['Entra ASA'],[254167079],[[318538]],2025-05-23,2025,2025/1940600983.txt,"HEADLINE= Entra ASA, Q1 2025 Earnings Call, Ap..."
23,1928977574,2025-02-19,Castellum AB (publ),883159,[222504],['Castellum AB (publ)'],[883159],[[222504]],2025-04-01,2025,2025/1928977574.txt,"HEADLINE= Castellum AB (publ), 2024 Earnings C..."
6,1940573869,2025-04-23,Corbion N.V.,874125,[100635],['Corbion N.V.'],[874125],[[100635]],2025-05-23,2025,2025/1940573869.txt,"HEADLINE= Corbion N.V., Q1 2025 Earnings Call,..."
71,1938390515,2025-04-08,Hilton Food Group plc,5542559,[284618],['Hilton Food Group plc'],[5542559],[[284618]],2025-04-23,2025,2025/1938390515.txt,"HEADLINE= Hilton Food Group plc, 2024 Earnings..."


#### Process single transcript

In [3]:
# Modify OpenAI's API key and API base to use vLLM's API server.

openai_api_key = "None"
openai_api_base = "http://sh03-18n07:12345/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,

)

transcript = df.iloc[0]['transcript']

tariffs_sys_prompt = """ 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are assisting me in analyzing companies' earnings calls. The transcript of the earnings call will be supplied as the user prompt. 

## Response Instructions: Part 1 (Analysis) ##

The first part of your response should be an analysis of whether the firm is reporting that its decisions are being affected by tariff policies. Tariffs are defined as taxes imposed on imported foreign goods. These must be imposed by the importing country in order to be classified as tariffs. Tariffs are not export restrictions, quotas, embargoes, financial sanctions, boycotts, or non-tariff barriers.

This part of your response should be enclosed between the tags <ANALYSIS> and </ANALYSIS>. Keep your analysis to 300 words or less. Make sure to cover all of the following points in your summary analysis:

- Whether tariffs are discussed explicitly or implicitly (e.g., by not using the word "tariffs" but referencing impacts on the firm's business that clearly relate to tariffs).
- Whether the firm discusses current tariffs (i.e., tariffs that have already been imposed) or the potential of future tariffs (i.e., tariffs that have not yet been imposed).
- Whether the firm discusses tariffs on the goods it sells, or on those that it buys.
- The nature and details of the tariff policies that are discussed.
- The countries that are imposing the tariffs and those that are subject to the tariffs.
- Any impacts on the firms' current profits.
- Any potential impact on the firm's future profits.
- Any impacts on the firms' behavior (e.g., in terms of investment, pricing, employment, inventory, R&D, project delay, supply chains, or other future plans).
- Any details on the geographies affected by the changes in the firms' behavior (e.g., if the firm reports importing less, report from which destination).

## Response Instructions: Part 2 (Structured JSON Output) ##

The second part of your response should be a structured output in JSON format that recaps your analysis in a structured way. This part of your response should be enclosed between the tags <JSON> and </JSON>. The JSON output must have the following fields exactly. Please make sure to enforce the JSON schema specified below strictly: i.e., the column names should correspond exactly to those listed below:

1. A boolean flag called "tariffs_any", which should be 1 if the firm discusses tariffs at any point in the call, and 0 otherwise. Even if the term "tariffs" is not explicitly used throughout the call, you should return a 1 if the firm discusses impacts on its business that clearly relate to tariffs.
2. A field called "countries_imposing_tariffs", listing the countries whose tariffs policy the firm discusses, if any. For example, if the firm reports concerns about tariffs imposed by the US government on goods imported from China, this field should say "USA"
3. A field called "countries_receiving_tariffs", listing the entities (countries or firms) targeted by the tariffs discussed by the firm, if any. For example, if the firm reports concerns about tariffs imposed by the US government on goods imported from China, this field should say "China".
4. A boolean flag called "investment_up", which should be 1 if the firm says it is increasing or planning to increase its investment as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs.
5. A boolean flag called "investment_down", which should be 1 if the firm says it is decreasing or planning to decrease its investment as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs.
6. A field called "countries_investment_up", listing the countries where the firm reports increasing or planning to increase investment as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
7. A field called "countries_investment_down", listing the countries where the firm reports decreasing or planning to decrease investment as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
8. A boolean flag called "domestic_investment_up", which should be 1 if the firm says it is increasing or planning to increase its investment in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs. 
9. A boolean flag called "domestic_investment_down", which should be 1 if the firm says it is decreasing or planning to decrease its investment in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs. 
10. A boolean flag called "labor_up", which should be 1 if the firm says it is increasing or planning to increase its number of workers or their hours worked as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
11. A boolean flag called "labor_down", which should be 1 if the firm says it is decreasing or planning to decrease its number of workers or their hours worked as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
12. A field called "countries_labor_up", listing the countries where the firm reports increasing or planning to increase its number of workers or their hours worked as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
13. A field called "countries_labor_down", listing the countries where the firm reports decreasing or planning to decrease its number of workers or their hours worked as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
14. A boolean flag called "domestic_labor_up", which should be 1 if the firm says it is increasing or planning to increase its number of workers or their hours worked in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
15. A boolean flag called "domestic_labor_down", which should be 1 if the firm says it is decreasing or planning to decrease its number of workers or their hours worked in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
16. A boolean flag called "sales_price_up", which should be 1 if the firm says it is increasing or planning to increase the price it charges for the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its product pricing decisions to tariffs. 
17. A boolean flag called "sales_price_down", which should be 1 if the firm says it is decreasing or planning to decrease the price it charges for the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its product pricing decisions to tariffs.
18. A field called "countries_sales_price_up", listing the countries where the firm reports increasing or planning to increase the price it charges for the products it sells as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
19. A field called "countries_sales_price_down", listing the countries where the firm reports decreasing or planning to decrease the price it charges for the products it sells as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
20. A boolean flag called "input_price_up", which should be 1 if the firm says it faces or expects to face higher prices on the inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its input prices to tariffs. 
21. A boolean flag called "input_price_down", which should be 1 if the firm says it faces or expects to face lower prices on the inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its input prices to tariffs.
22. A field called "countries_input_price_up", listing the countries where the firm reports facing or expecting to face higher prices on the inputs it purchases as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
23. A field called "countries_input_price_down", listing the countries where the firm reports facing or expecting to face lower prices on the inputs it purchases as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
24. A boolean flag called "inventory_product_up", which should be 1 if the firm says it is increasing or expecting to increase the amount of inventory of the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs. 
25. A boolean flag called "inventory_product_down", which should be 1 if the firm says it is decreasing or expecting to decrease the amount of inventory of the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs.
26. A boolean flag called "inventory_input_up", which should be 1 if the firm says it is increasing or expecting to increase the amount of inventory of inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs. 
27. A boolean flag called "inventory_input_down", which should be 1 if the firm says it is decreasing or expecting to decrease the amount of inventory of inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs.
28. A boolean flag called "rd_up", which should be 1 if the firm says it is increasing or expecting to increase the amount of research and development it undertakes as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its research and development decisions to tariffs. 
29. A boolean flag called "rd_down", which should be 1 if the firm says it is decreasing or expecting to decrease the amount of research and development it undertakes as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its research and development decisions to tariffs.
30. A boolean flag called "delay", which should be 1 if the firm says it is delaying a decision as a result of current or future tariffs. Only set the flag to 1 if the company explicitly attributes the delay to tariffs.
31. A boolean flag called "supply_chain_adj", which should be 1 if the firm says it is adjusting its supply chain as a result of current or future tariffs. Only set the flag to 1 if the company is explicitly adjusting supply chains due to tariffs.
32. A field called "countries_supply_chain_adj_towards", listing the countries where the firm reports increasing the exposure of its supply chain as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
33. A field called "countries_supply_chain_adj_away", listing the countries where the firm reports reducing the exposure of its supply chain as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
34. A boolean flag called "lobbying_up", which should be 1 if the firm says it is increasing or expecting to increase its lobbying activity as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its lobbying policy to tariffs. 
35. A boolean flag called "lobbying_down", which should be 1 if the firm says it is decreasing or expecting to decrease its lobbying activity as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its lobbying policy to tariffs.
36. A boolean flag called "profit_margin_up", which should be 1 if the firm says its profit margins have increased or are expected to increase as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its profit margins to tariffs. 
37. A boolean flag called "profit_margin_down", which should be 1 if the firm says its profit margins have decreased or are expected to decreased as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its profit margins to tariffs.
38. A boolean flag called "financing_adj", which should be 1 if the firm says it is adjusting its financing decisions in response to current or future tariffs. Only set the flag to 1 if the company is explicitly adjusting its financing decisions due to tariffs.

## Response Instructions: Part 3 (Summary) ##
Write a single summary of 100 words or fewer that captures only how the firm is (or may be) affected by tariffs. Omit all unrelated content. If the firm is not affected by any tariffs, write "Not affected." Enclose this analysis between the tags <SUMMARY> and </SUMMARY>.

## Response Instructions: Part 4 (Evaluation) ##
The fourth part of your response should be an evaluation of how well the JSON structured summary agrees with your initial analysis. Keep the evaluation to 100 words or less. This part of your response should be enclosed between the tags <EVAL> and </EVAL> and be 100 words or less.

## Important Notes ##
- Do not consider restrictions on exports (imposed by the seller country) or generic financial sanctions as indications of tariffs. Tariffs must be taxes imposed by the importing country.
<|end_of_text|>
"""

tariffs_user_prompt = '''
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Analyze the tariffs information from the transcript below:
<|start_transcript_id|>
{}
<|end_transcript_id|>
<|end_of_text|> '''.format(transcript)

chat_response = client.chat.completions.create(
    model="/scratch/groups/maggiori/raw_model_weights/Llama-3.3-70B-Instruct-AWQ",
    messages=[
        {"role": "system", "content": tariffs_sys_prompt},
        {"role": "user", "content": tariffs_user_prompt},
    ],
    max_tokens=2048,
    temperature=0
)

llm_output = chat_response.choices[0].message.content
print("Chat response:\n", llm_output)

2025-09-05 11:12:59,178 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/chat/completions "HTTP/1.1 200 OK"


Chat response:
 <ANALYSIS>
The transcript of the earnings call does not explicitly mention tariffs. However, there is a brief discussion about tariffs in the context of the current market environment and their potential impact on the portfolio. The CEO mentions that the company is "relatively insulated" from movements in foreign exchange rates and tariffs due to the construction of the portfolio, which is primarily focused on U.S. borrowers. The CEO also notes that the company has done work to assess potential tariff vulnerability and does not believe they have a meaningful exposure on a first-order basis. However, the CEO cautions that the second, third, and fourth-order impacts of tariffs are uncertain and could potentially affect the portfolio.

The discussion of tariffs is implicit, and the company does not provide specific details about the nature of the tariffs, the countries imposing them, or the countries subject to them. The CEO's comments suggest that the company is monitorin

In [4]:
cols = [  
    "tariffs_any",
    "countries_imposing_tariffs",
    "countries_receiving_tariffs",
    "investment_up",
    "investment_down",
    "countries_investment_up",
    "countries_investment_down",
    "domestic_investment_up",
    "domestic_investment_down",
    "labor_up",
    "labor_down",
    "countries_labor_up",
    "countries_labor_down",
    "domestic_labor_up",
    "domestic_labor_down",
    "sales_price_up",
    "sales_price_down",
    "countries_sales_price_up",
    "countries_sales_price_down",
    "input_price_up",
    "input_price_down",
    "countries_input_price_up",
    "countries_input_price_down",
    "inventory_product_up",
    "inventory_product_down",
    "inventory_input_up",
    "inventory_input_down",
    "rd_up",
    "rd_down",
    "delay",
    "supply_chain_adj",
    "countries_supply_chain_adj_towards",
    "countries_supply_chain_adj_away",
    "lobbying_up",
    "lobbying_down",
    "profit_margin_up",
    "profit_margin_down",
    "financing_adj",
    "parse_error"]

# Function to safely parse JSON and handle errors
def safe_json_loads(output_str):
    try:
        result = {}
        parse_error = False
        # Remove the header/footer tags
        output_str = output_str.replace("<|start_header_id|>assistant<|end_header_id|>", "")
        output_str = output_str.replace("```json", "")
        output_str = output_str.replace("```", "</JSON>")
        
        # Extract JSON portion between <JSON> and </JSON>
        json_start = output_str.find("<JSON>") + len("<JSON>")
        json_end = output_str.find("</JSON>")
        if json_start == -1 or json_end == -1:
            parse_error = True
        else:
            json_str = output_str[json_start:json_end]
            result = json.loads(json_str)
        
        # Extract analysis text between <ANALYSIS> and </ANALYSIS>
        analysis_start = output_str.find("<ANALYSIS>") + len("<ANALYSIS>")
        analysis_end = output_str.find("</ANALYSIS>")
        if analysis_end == -1:
            analysis_end = output_str.find("<JSON>")
        if analysis_start == -1 or analysis_end == -1:
            parse_error = True
        else:
            result['analysis'] = output_str[analysis_start:analysis_end].strip()
            
        # Extract analysis text between <SUMMARY> and </SUMMARY>
        summary_start = output_str.find("<SUMMARY>") + len("<SUMMARY>")
        summary_end = output_str.find("</SUMMARY>")
        if summary_end == -1:
            summary_end = output_str.find("<EVAL>")
        if summary_start == -1 or summary_end == -1:
            parse_error = True
        else:
            result['summary'] = output_str[summary_start:summary_end].strip()
            
        eval_start = output_str.find("<EVAL>") + len("<EVAL>") 
        eval_end = output_str.find("</EVAL>")
        if eval_start == -1 or eval_end == -1:
            parse_error = True
        else:
            result['evaluation'] = output_str[eval_start:eval_end].strip()
        
        result['parse_error'] = "1" if parse_error else "0"
        
        return result
    except:
        return None
    
json_result = safe_json_loads(llm_output)
print(json.dumps(json_result, indent=2))

{
  "tariffs_any": 1,
  "countries_imposing_tariffs": "NaN",
  "countries_receiving_tariffs": "NaN",
  "investment_up": 0,
  "investment_down": 0,
  "countries_investment_up": "NaN",
  "countries_investment_down": "NaN",
  "domestic_investment_up": 0,
  "domestic_investment_down": 0,
  "labor_up": 0,
  "labor_down": 0,
  "countries_labor_up": "NaN",
  "countries_labor_down": "NaN",
  "domestic_labor_up": 0,
  "domestic_labor_down": 0,
  "sales_price_up": 0,
  "sales_price_down": 0,
  "countries_sales_price_up": "NaN",
  "countries_sales_price_down": "NaN",
  "input_price_up": 0,
  "input_price_down": 0,
  "countries_input_price_up": "NaN",
  "countries_input_price_down": "NaN",
  "inventory_product_up": 0,
  "inventory_product_down": 0,
  "inventory_input_up": 0,
  "inventory_input_down": 0,
  "rd_up": 0,
  "rd_down": 0,
  "delay": 0,
  "supply_chain_adj": 0,
  "countries_supply_chain_adj_towards": "NaN",
  "countries_supply_chain_adj_away": "NaN",
  "lobbying_up": 0,
  "lobbying_down"

#### Batch processing with ASYNC

In [5]:
def prepare_prompt(x):
    out =  f'''{tariffs_sys_prompt}\n<|begin_of_text|><|start_header_id|>user<|end_header_id|>
    Analyze the tariffs information from the transcript below:
    <|start_transcript_id|>{x}<|end_transcript_id|>
    <|end_of_text|> '''
    return out

prompts = df['transcript'].apply(prepare_prompt)

async_client = AsyncOpenAI(
    base_url="http://sh03-18n07:12345/v1",
    api_key="None"
)

async def process_text(prompt):
    try:
        response = await async_client.completions.create(
            model="/scratch/groups/maggiori/raw_model_weights/Llama-3.3-70B-Instruct-AWQ",
            prompt=prompt,
            max_tokens=2048,
            temperature=0,
            timeout=300
        )
        return response.choices[0].text if len(response.choices) > 0 else ""
    except Exception as e:
        print(f"Error: {e}")
        return ""

async def process_batch(prompts, batch_size = 10):
    results = []
    
    # Process in batches
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size]
        
        # Create tasks for concurrent execution
        tasks = [
            process_text(prompt) for prompt in batch
        ]
        
        # Execute batch concurrently and wait for all to complete
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)
        
        print(f"Completed batch {i//batch_size + 1}, processed {len(results)}/{len(prompts)} items")

    return results

results = await process_batch(list(prompts.values))

2025-09-05 11:14:28,861 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:35,091 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:38,588 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:40,712 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:42,036 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:43,545 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:50,881 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:51,806 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:14:51,960 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:15:07

Completed batch 1, processed 10/20 items


2025-09-05 11:16:34,193 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:35,224 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:35,348 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:36,775 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:41,134 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:41,576 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:48,939 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:53,129 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:55,019 - INFO - HTTP Request: POST http://sh03-18n07:12345/v1/completions "HTTP/1.1 200 OK"
2025-09-05 11:16:55

Completed batch 2, processed 20/20 items


In [6]:
df_tariffs = df.copy()
parsed_outputs = [safe_json_loads(output) for output in results]
all_dicts = []
for parsed_output in parsed_outputs:
    output_dict = {}
    for col in cols:
        output_dict[col] = str(parsed_output.get(col)) if isinstance(parsed_output, dict) else None
    if parsed_output is None:
        output_dict['parse_error'] = "1"
    all_dicts.append(output_dict)
    
for col in all_dicts[0].keys():
    df_tariffs[col] = [d[col] for d in all_dicts]
num_parse_error = df_tariffs[df_tariffs['parse_error'] == '1'].shape[0]
if num_parse_error > 0:
    print(f"Number of rows with parse error: {num_parse_error} ")
    
df_tariffs = df_tariffs[df_tariffs["tariffs_any"] == '1']
df_tariffs.head()

,key,date,companyname_primary,companyid_primary,gvkey_primary,companynames,companyids,gvkeys,version_date,year,...,delay,supply_chain_adj,countries_supply_chain_adj_towards,countries_supply_chain_adj_away,lobbying_up,lobbying_down,profit_margin_up,profit_margin_down,financing_adj,parse_error
35,1915209450,2025-02-05,"Golub Capital BDC, Inc.",81683203,[184689],"['Golub Capital BDC, Inc.']",[81683203],[[184689]],2025-04-01,2025,...,0,0,NaN,NaN,0,0,0,0,0,0
36,1940600983,2025-04-29,Entra ASA,254167079,[318538],['Entra ASA'],[254167079],[[318538]],2025-05-23,2025,...,0,0,NaN,NaN,0,0,0,0,0,0
6,1940573869,2025-04-23,Corbion N.V.,874125,[100635],['Corbion N.V.'],[874125],[[100635]],2025-05-23,2025,...,0,1,NaN,China,0,0,0,0,0,0
71,1938390515,2025-04-08,Hilton Food Group plc,5542559,[284618],['Hilton Food Group plc'],[5542559],[[284618]],2025-04-23,2025,...,0,0,NaN,NaN,0,0,0,0,0,0
8,1936868056,2025-05-08,Budweiser Brewing Company APAC Limited,620677159,[332813],['Budweiser Brewing Company APAC Limited'],[620677159],[[332813]],2025-05-23,2025,...,0,0,NaN,NaN,0,0,0,0,0,0
